# Project Title
## Data Engineering Project
### NYC Citi Bike Sharing Optimization Application

#### Project Summary
Bike and scooter sharing services have taken the world by storm in the past few years. It is a service that has companies, governments, and people interested in moving our cities and communities into a more sustainable enviroments. The aim of many stakeholders is to encourage everyone to get to their destinations utilizing bikes and electric scooters. This nobel innovation is a service to our earth to fight global warming, to our society to be more enviromentally conscious, and to our communities to be more sustainable. Unfortunately, bike and scooter sharing have faced dangerous problems with cities, like New York City, not ready to provide safe routes or zoned parking. Other problems occur on accessibility and optimization of providers of this service like Uber, Lyft, and Citi Bikes. I believe that a unified source of bike data from providers, government, and users is an important step to allow us to start solving these problems. This project will serve as a first step to create mentioned data source.

#### Scope the Project and Data Gathering

The project orchestrate collection of data from major sources in the bike sharing space. The goal is to pull relevant information to centralize potentially relevant information to solve current and future problems that arise from providing this service to customers. Data would be structured in a clean but most raw version in order to keep a wide scoop for usage by data analysts, business analysts, data scientist, government officials, city planners, and many more. The main goal is to unify the dispersed data from major providers like Lyft, Uber, Citi Bikes, and city-based data on traffic, bike routes, and other data from the NYC Open Data project. 

The scope of the project will be limited to NYC for location with a potential to implement in other cities after its success here. The time scope of the project will be looking at data starting from June 2013 and going up to August 2020 with data pipelines set to run on a monthly, quarterly and annually timeframe depending on the data sources.

The data sources are as follows:

    1. **City Bike:** The data from the NYCBS project, the data is provided in an [Amazon S3 buckets](https://s3.amazonaws.com/tripdata/index.html). It is split into seperate zip files for every month from 2013 to August 2020 -for the purposes of this project we will focus on data starting January 2018 - with an added zip file to the same bucket every month.
    
    2. **NYC OpenData:** The data from NYC's OpenData project would be rertrieved through an API endpoints for data for the city's [Bicycle Routes](https://data.cityofnewyork.us/Transportation/Bicycle-Routes/7vsa-caz7).
    
The tools that will be used in this project are Apache Cassandra, Apache Spark, Amazon's S3 and RedShift, and Apache Airflow. Given that we will be working with large amounts of data from multiple sources, we will work with Cassandra and optimize our reads with queries needed to setup the data of interest in mind. Fast writes will be essential to have our data available for later use by different types of users. Apache Spark is the tool that will be oriented towards handling potential data streams from a real-time data source like Google's Map Platform. Although for the purposes of this project, we will be running the data lake with Spark on local mode, the feature of upgrading our system to a manager mode for potential future use by personnel from poviders, goverment, and any professional who is curious about this unified source of data on bike sharing in NYC. AWS's S3 is a tool that is needed to work with when extracting data from sources like Lyft and Citi Bike. In addition, RedShift is the tool that will be utilized as the home of our data as a data warehouse. Finally, Airflow is our data management tool using DAGs that will organize the schedules of data pulls for a continued and updated versions of our unified data sources.